## SDP examples for *Rotational covariance restricts available quantum state*
Fynn Otto, Konrad Szymański

This file contains the calculations behind the Examples 3 and 4 of our paper, demonstrating the use of semidefinite optimization for determination of maximal fidelity achievable with rotationally (SU(2)) covariant transformations. First, let us import the libraries used in the code (numpy for array manipulations, mosek and picos for semidefinite optimization and scipy function for Wigner 3J symbols).

In [8]:
import numpy as np
import mosek
import picos as pc
from sympy.physics.wigner import wigner_3j
ATOL = 1E-8
# IMPORANT: If MOSEK license is not automatically found, uncomment and update the following line.
#%env MOSEKLM_LICENSE_FILE=/path/to/mosek/mosek.lic

Now, the main functions. Some, like **kb**, **pos** and **rhoij** are helper ones, used to determine the indices of vectors and density operators from quantum numbers. The function **K** defines the effect of a single Kraus operator on an input density operator, parameterized by weights stored in its argument **Hmat**, **channel** returns a complete result summed over Kraus operators. Then, **getSDP** builds an instance of semidefinite optimization based on input and target states -- the goal is to find an SU(2)-covariant channel (parameterized with SU(2)-covariant Kraus operators) maximizing the fidelity between transformed input state and the target.

In [39]:
def kb(a,jlim):
    """Creates a ket-bra operator |`a[1]`/2><`a[2]`/2| on a system with max. spin `jlim` """
    
    kb = np.zeros((int(jlim**2/2 + 3/2*jlim + 1), int(jlim**2/2 + 3/2*jlim + 1)))
    kb[a] += 1
    
    return kb

def pos(jp,j,J):
    """Maps a tuple of spin numbers `jp` and `j` to an integer given the parameter `J` related to a Kraus operator """
    
    x = 0
    if jp<=J:
        x = jp**2/2+jp/2+(j+jp-J)/2
    else:
        x = (J+1)*(J+2)/2+(jp-J-1)*(J+1)+(j+J-jp)/2
    
    return int(x)

def rhoij(j,mj,k,mk):
    """Returns the index of the density matrix element |`j`/2,`mj`/2><`k`/2,`mk`/2|"""
    
    if (j-mj)%2 == 1 or (k-mk)%2 == 1:
        raise ValueError("No valid arguments given. They do not match spin states.")
    
    return (int(j**2/2+j/2+(j-mj)/2),int(k**2/2+k/2+(k-mk)/2))

def vecstate(j,mj,jlim):
    idx=int(j**2/2+j/2+(j-mj)/2)
    ret=np.zeros(int(jlim**2/2 + 3/2*jlim + 1))
    ret[idx]=1
    return ret

def projector(ket):
    return np.outer(ket,ket.conjugate())

def matprint(mat, fmt="g"): #from https://gist.github.com/braingineer/d801735dac07ff3ac4d746e1f218ab75
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

def K(J, M, jlim, Hmat, rho):#=np.eye(6)):
    """Returns the product K_{`J`,`M`} rho K_{`J`,`M`}^\dagger depending on the parameters defined in `Hmat`"""
    
    sig = np.zeros_like(rho)
    for jp in range(jlim+1):
        for m in range(-jp,jp+2,2):
            for j in range(abs(J-jp),min(J+jp,jlim)+1,2):
                for kp in range(jlim+1):
                    for l in range(-kp,kp+2,2):
                        for k in range(abs(J-kp),min(J+kp,jlim)+1,2):
                            if abs(m-M)<=j and abs(l-M)<=k:
                                x=(-1)**((jp+kp-m-l)/2)*float(wigner_3j(jp/2,J/2,j/2,-m/2,M/2,(m-M)/2))*float(wigner_3j(kp/2,J/2,k/2,-l/2,M/2,(l-M)/2))*rho[rhoij(j,m-M,k,l-M)]*kb(rhoij(jp,m,kp,l),jlim)*Hmat[pos(jp,j,J),pos(kp,k,J)]
                                sig = sig + x
    
    return sig

def channel(jmax, jlim, Hmat, rho):
    """Returns the output of the channel characterized by `Hmat` given the input `rho`"""

    state = np.zeros_like(rho)
    for j in range(jmax+1):
        for m in range(-j,j+2,2):
            state = state + K(j,m,jlim,Hmat[j],rho)
    return state

def paramLabels(Jmax,jlim):
    
    positions=[]
    for h in range(0,Jmax+1):
        posh=[]
        for jp in range(jlim+1):
            for j in range(abs(h-jp),min(h+jp,jlim)+1,2):
                posh.append([pos(jp,j,h),jp,j])
        #print(posh)
        positions.append(posh)
    
    return positions

def getSDP(rho, target, jlim):
    
    rho = np.array(rho)
    target = np.array(target)

    # Make sure that both rho and target are valid quantum states
    assert(rho.shape==target.shape), "Please use two states on the same Hilbert space"
    assert(np.isclose(np.trace(rho),1,ATOL) and np.array_equal(rho,rho.conjugate().T) and np.all(np.linalg.eigvals(rho)>=-ATOL)), "The first input is not a quantum state."
    assert(np.isclose(np.trace(target),1,ATOL) and np.array_equal(target,target.conjugate().T) and np.all(np.linalg.eigvals(target)>=-ATOL)), "The second input is not a quantum state."

    a = paramLabels(2*jlim,jlim)
    
    SDP = pc.Problem()

    X = pc.ComplexVariable("X",rho.shape)

    # Set up parameter matrices with the constraint of being positive semidefinite
    Ha = []
    for j,b in enumerate(a):
        Ha.append(pc.HermitianVariable(f'H_{j}', int(np.amax(np.transpose(b)[0])+1)))#len(b))))
    SDP.add_list_of_constraints([h >> 0 for h in Ha])

    # Add the normalization constraint for the channel to be trace-preserving
    constraints = [[np.diag([x in np.transpose(a[i])[0] and a[i][np.where(np.transpose(a[i])[0]==x)[0][0]][2]==j for x in range(int(np.amax(np.transpose(a[i])[0])+1))]) for i in range(len(a))] for j in range(jlim+1)]
    SDP.add_list_of_constraints([sum([Ha[i][k,k] for i in range(len(Ha)) for k in range(int(np.sqrt(len(Ha[i])))) if const[i][k][k]])==j+1 for j,const in enumerate(constraints)])

    # Constraint for the fidelity SDP
    SDP.add_constraint(pc.block([[target, X],[X.H, channel(2*jlim, jlim, Ha, rho)]]) >> 0)

    # Define objective function to maximize
    SDP.set_objective("max", 1/2*pc.trace(X+X.H))

    return(SDP)


In order to make programming more straightforward, most of the functions take as their inputs double the total spin $2j$ and magnetic spin quantum number $2m$ instead of $j$ and $m$. For instance, the state $\ket{j=\frac32,m=\frac32}$ corresponds in code to **vecstate(j=3,mj=3,jlim)**. 

Thus, the following code recreates the Example 4: the **inputState** variable is density matrix representation of $\ket{j=\frac32,m=\frac32}$, and **outputState** corresponds to $\ket{j=2,m=2}$. Evidently, the optimal state is a probabilistic mixture of $\ket{j=2,m=2}$ with weight $\frac45$ and $\ket{j=2,m=1}$ with weight $\frac15$.

In [42]:
# Define upper limit for the representation of SU(2)
jlim = 4

# Define input and target state
inputState = projector(vecstate(3,3,jlim))#kb(rhoij(3,3,3,3),jlim)#1/2*(kb(rhoij(0,0,0,0),2)+kb(rhoij(2,0,2,0),2)+kb(rhoij(0,0,2,0),2)+kb(rhoij(2,0,0,0),2))
targetState = projector(vecstate(4,4,jlim))#kb(rhoij(4,4,4,4),jlim)

# SDP to determine the maximal fidelity
SDP = getSDP(inputState,targetState,jlim)
SDP.solve(solver="mosek", verbosity=0)

# Print the parameter matrices for the channel to achieve the maximum fidelity, the fidelity, and the channels output
xfinal = SDP.get_variable("X").value
fid = np.trace(xfinal+xfinal.H)/2
Ha = [SDP.get_variable(f'H_{j}').value for j in range(len(SDP.mutables)-1)]


print("maximal output fidelity:", np.real(fid))
print("optimal output state maximizing fidelity with the target:")
matprint(np.real(np.round(channel(2*jlim,jlim,Ha, inputState),decimals=1)))



maximal output fidelity: 0.8944816581290255
optimal output state maximizing fidelity with the target:
 0  0  0   0  -0   0  0  0  0  0    0    0  -0   0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0   0  0  0  0  0    0   -0   0   0  0  
-0  0  0   0   0   0  0  0  0  0    0    0  -0   0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0  -0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0   0  0  0  0  0  0.8    0   0   0  0  
 0  0  0  -0   0   0  0  0  0  0    0  0.2   0   0  0  
-0  0  0   0  -0   0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0  -0  0  0  0  0    0    0   0   0  0  
 0  0  0   0   0   0  0  0  0  0    0    0   0   0  0  


The Example 3 follows a similar pattern. The input state is $\frac1{\sqrt6}\left(\ket{j=0,m=0}+\ket{j=1,m=0}+2\ket{j=\frac32,m=\frac32}\right)$, and we are looking for a SU(2)-covariant channel transforming it to a state maximizing the fidelity with $\ket{j=\frac12,m=-\frac12}$. The fidelity is numerically indistinguishable from $\sqrt{\frac{13}{15}}\approx 0.93$

In [50]:
# Define upper limit for the representation of SU(2)
jlim = 3

# Define input and target state

inputState=projector(1/np.sqrt(6)*(vecstate(0,0,jlim)+vecstate(2,0,jlim)+2*vecstate(3,3,jlim)))
targetState = projector(vecstate(1,-1,jlim))#kb(rhoij(1,-1,1,-1),jlim)

# SDP to determine the maximal fidelity
SDP = getSDP(inputState,targetState,jlim)
SDP.solve(solver="mosek", verbosity=0)
# Print the parameter matrices for the channel to achieve the maximum fidelity, the fidelity, and the channels output
xfinal = SDP.get_variable("X").value
fid = np.trace(xfinal+xfinal.H)/2
Ha = [SDP.get_variable(f'H_{j}').value for j in range(len(SDP.mutables)-1)]


print("maximal output fidelity:", np.real(fid))
print("optimal output state maximizing fidelity with the target:")
matprint(np.real(np.round(channel(2*jlim,jlim,Ha, inputState),decimals=3)))


maximal output fidelity: 0.9309649175363803
optimal output state maximizing fidelity with the target:
 0      0      0   0   0   0  -0   0   0   0  
 0  0.133      0   0   0  -0   0  -0   0   0  
 0      0  0.867  -0   0   0   0   0  -0   0  
 0      0     -0   0   0   0   0   0  -0   0  
 0      0      0   0   0   0   0   0   0  -0  
 0     -0      0   0   0   0   0   0   0   0  
-0      0      0   0   0   0   0   0   0   0  
 0     -0      0   0   0   0   0   0   0   0  
 0      0     -0  -0   0   0   0   0   0   0  
 0      0      0   0  -0   0   0   0   0   0  
